# 환경설정

In [6]:
# [Cell 1] 환경설정 및 라이브러리 임포트
import os
import gc
import re
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration, BitsAndBytesConfig

# 메모리 파편화 방지 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def clean_memory():
    """VRAM 메모리 캐시를 비우는 함수"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

clean_memory()
print("1단계: 라이브러리 임포트 및 환경설정 완료!")

1단계: 라이브러리 임포트 및 환경설정 완료!


# 전처리

In [7]:
# [Cell 2] 전처리 함수 정의 (OpenCV)

def order_points(pts):
    """좌표 정렬 (좌상, 우상, 우하, 좌하 순서)"""
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    """투시 변환 (찌그러진 사각형 펴기)"""
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (maxWidth, maxHeight))

def dewarp_book_page(img, curve_intensity=20):
    """책 곡면 보정"""
    rows, cols = img.shape[:2]
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    dst = np.zeros((rows, cols, 3), dtype="uint8")
    alpha = np.radians(curve_intensity)
    r = cols / alpha 
    cy, cx = rows / 2, cols / 2
    y_indices, x_indices = np.indices((rows, cols))
    x_map = r * np.sin((x_indices - cx) / r) + cx
    y_map = (y_indices - cy) * (r / np.sqrt(np.square(r) - np.square(x_indices - cx))) + cy
    dst = cv2.remap(img, x_map.astype(np.float32), y_map.astype(np.float32), 
                    interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    return dst

def expand_contour(cnt, scale=1.03):
    """윤곽선 영역 살짝 확장"""
    M = cv2.moments(cnt)
    if M['m00'] == 0:
        return cnt
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cnt_norm = cnt - [cx, cy]
    cnt_scaled = cnt_norm * scale
    cnt_new = cnt_scaled + [cx, cy]
    return cnt_new.astype(np.int32)

def preprocess_document(image_path):
    """
    [핵심 함수] 이미지 경로를 받아 전처리(Crop -> Dewarp) 후 PIL Image 객체 반환
    """
    if not os.path.exists(image_path):
        print(f"파일 없음: {image_path}")
        return None

    cv_image = cv2.imread(image_path)
    if cv_image is None: return None
    
    orig = cv_image.copy()
    ratio = cv_image.shape[0] / 500.0
    h = 500
    w = int(cv_image.shape[1] / ratio)
    image_resized = cv2.resize(cv_image, (w, h))

    # 윤곽선 검출
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 50, 150)
    kernel = np.ones((5,5), np.uint8)
    edged = cv2.dilate(edged, kernel, iterations=1)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

    screenCnt = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            screenCnt = approx
            break

    # 윤곽선 못 찾으면 원본 사용
    if screenCnt is None:
        print(">> 문서 윤곽선을 찾지 못해 원본 이미지를 사용합니다.")
        return Image.fromarray(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))

    # 변환 적용
    screenCnt = expand_contour(screenCnt, scale=1.03)
    warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
    final_result = dewarp_book_page(warped, curve_intensity=15)
    
    print(">> 전처리(문서 영역 추출 및 보정) 성공!")
    return Image.fromarray(cv2.cvtColor(final_result, cv2.COLOR_BGR2RGB))

print("2단계: 전처리 함수 준비 완료!")

2단계: 전처리 함수 준비 완료!


# VARCO VISION 2.0

In [8]:
# [Cell 3] 모델 로드 및 OCR 실행

# 1. 모델 설정 (이미 로드되어 있으면 건너뜀)
if 'model' not in locals():
    print("모델 로딩 중...")
    model_id = "NCSOFT/VARCO-VISION-2.0-1.7B-OCR"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    model = LlavaOnevisionForConditionalGeneration.from_pretrained(
        model_id, quantization_config=bnb_config, device_map="auto", attn_implementation="sdpa", low_cpu_mem_usage=True
    )
    processor = AutoProcessor.from_pretrained(model_id)
    print("모델 로드 완료!")

def clean_ocr_text(text):
    cleaned = re.sub(r'\d+\.\d+,\s*\d+\.\d+,\s*\d+\.\d+,\s*\d+\.\d+', '', text)
    return re.sub(r'\s+', ' ', cleaned).strip()

# ==========================================
# [입력] 처리할 파일명 입력
# ==========================================
target_file_path = "img1.jpg"  # <--- 여기를 수정하세요
# ==========================================

if os.path.exists(target_file_path):
    try:
        # [Step 1] 전처리 함수 호출
        print(f"[{target_file_path}] 처리 시작...")
        image = preprocess_document(target_file_path) # 위에서 만든 함수 사용
        
        # [Step 2] 모델 입력 준비
        target_size = 512
        w, h = image.size
        if max(w, h) > target_size:
            scale = target_size / max(w, h)
            image = image.resize((int(w * scale), int(h * scale)), resample=Image.LANCZOS)

        conversation = [
            {"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": "<ocr>"}]}
        ]
        inputs = processor.apply_chat_template(
            conversation, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"
        ).to(model.device)

        # [Step 3] 추론
        with torch.inference_mode():
            generate_ids = model.generate(**inputs, max_new_tokens=1024, do_sample=False, use_cache=True)

        full_output = processor.decode(generate_ids[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
        
        print(f"\n{'='*20} OCR 결과 {'='*20}")
        print(clean_ocr_text(full_output))
        print(f"{'='*50}")

    except Exception as e:
        print(f"에러 발생: {e}")
    finally:
        del inputs, generate_ids, image
        clean_memory()
else:
    print("파일을 찾을 수 없습니다.")

[img1.jpg] 처리 시작...
>> 전처리(문서 영역 추출 및 보정) 성공!


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



==================== OCR 결과 ====================
일 수 요 억 오늘은 크레어 셋 을했다. 오늘은 은데 크레어 셋을했다. 오늘은 레모리 이션은 피구다. 그 리고남자, 여자팀으로 했다.
